In [1]:
% matplotlib inline
from bokeh.plotting import figure, save
import pandas as pd
from bokeh.plotting import figure, show, output_notebook

In [2]:
import pandas as pd
from bokeh.models import HoverTool
from bokeh.plotting import figure
from bokeh.palettes import Viridis256 as palette
from bokeh.models import LinearColorMapper
from bokeh.models import ColumnDataSource

In [3]:
import pysal as ps
from shapely.geometry import Point, Polygon
import numpy as np
import shapefile
from shapefile import *
import shapely

In [4]:
indmap=shapefile.Reader("myshapefile.shp")

In [5]:
states = set([i[0] for i in indmap.iterRecords()])

In [6]:
from bokeh.plotting import *
output_file("ElectionProj.html")

In [7]:
TOOLS="pan,wheel_zoom,box_zoom,reset,previewsave"
f= figure(title="General Information of General Election 2014", tools=TOOLS, plot_width=850, plot_height=800, x_axis_label='Longitude', y_axis_label='Latitude')

In [8]:
shapeObj = indmap.shapes()

In [9]:
def getParts ( shapeObj ):

    points = []

    num_parts = len( shapeObj.parts )
    end = len( shapeObj.points ) - 1
    segments = list( shapeObj.parts ) + [ end ]

    for i in range( num_parts ):
        points.append( shapeObj.points[ segments[i]:segments[i+1] ] )

        return points

In [10]:
def getDict ( state_name, shapefile ):

    stateDict = {state_name: {} }

    rec = []
    shp = []
    points = []
    
    for i in shapefile.shapeRecords( ):

        if i.record[0] == state_name:
            rec.append(i.record)
            shp.append(i.shape)
            
            #total_area = sum( [float(i[0]) for i in rec] ) / (1000*1000)
            
            for j in shp:
                for i in getParts(j):
                    points.append(i)
                
            lat = []
            lng = []
            for i in points:
                lat.append( [j[0] for j in i] )
                lng.append( [j[1] for j in i] )
                
            stateDict[state_name]['lat_list'] = lat
            stateDict[state_name]['lng_list'] = lng
            #stateDict[state_name]['total_area'] = total_area
            
            return stateDict

In [11]:
for state_name in states:
    data = getDict(state_name, indmap)
    f.patches(data[state_name]['lat_list'], data[state_name]['lng_list'], line_color="#000000", fill_color=
 '#ffffcc')

In [12]:
india=pd.read_csv('Ind_adm2_Points.csv')

In [13]:
indplo = india.pivot_table(values=['Latitude','Longitude'],index=['State']).copy()

In [14]:
df4 = pd.read_csv('GE2014_detail.csv')

In [15]:
indplo.reset_index(inplace=True)

In [16]:
indplo['State'][0]='Andaman & Nicobar Islands'
indplo['State'][6]='Chattisgarh'
indplo['State'][7]='Dadra & Nagar Haveli'
indplo['State'][8]='Daman & Diu'
indplo['State'][9]='NCT OF Delhi'
indplo['State'][14]='Jammu & Kashmir'
indplo['State'][25]='Odisha'
indplo['State'][33]='Uttarakhand'

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4:

In [17]:
indplo.rename(columns={'State':'STATE'},inplace=True)

In [18]:
disp = pd.merge(indplo,df4,on='STATE', how='outer')

In [19]:
del disp['FtoM_ratio']

In [20]:
disp.columns

Index(['STATE', 'Latitude', 'Longitude', 'F', 'M', 'O', 'Total voters',
       'Total_Electors', 'POLL_PERC in %'],
      dtype='object')

In [21]:
col = disp.pivot_table(values=['Latitude','Longitude','F', 'M', 'O', 'Total voters',
       'Total_Electors', 'POLL_PERC in %'],index=['STATE']).copy()

In [22]:
psource = ColumnDataSource(col)

In [23]:
from bokeh.models import HoverTool

In [24]:
my_hover = HoverTool()

In [25]:
my_hover.tooltips = [('State Name:',"@{STATE}"),('Number of Female Contestants in election:','@{F}'),('Number of Male Contestants in election:','@{M}'),('Number of Other Contastants in election:','@{O}'),('Number of registered voters:','@{Total_Electors}'),('Number of people voted:','@{Total voters}'),('Voter turnout:','@{POLL_PERC in %}%')]

In [26]:
f.add_tools(my_hover)

In [27]:
f.title.text_font_size = '20pt'
f.xaxis.axis_label_text_font_size = '15pt'
f.yaxis.axis_label_text_font_size = '15pt'

In [28]:
from bokeh.models import ColumnDataSource, Range1d, LabelSet, Label

In [29]:
labels = LabelSet(x='Longitude', y='Latitude', text='STATE', level='glyph',text_font_size='15pt',
              x_offset=5, y_offset=5, source=psource, render_mode='canvas')

In [30]:
f.circle('Longitude', 'Latitude', source=psource, color='red', size=6.5)

GlyphRenderer(id='19e4ae63-0a7a-4178-aac5-503cfa62ac09', ...)

In [31]:

f.add_layout(labels)

In [32]:
show(f)